# Проект с использованием SQL.

## 1 Описание проекта.

**Общая информация о заказчике:** Стартап-компания купила крупный сервис для чтения книг по подписке. 
В нашем распоряжение база данных, в ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Цель проекта** — проанализировать базу данных и сформулировать ценностное предложение для нового продукта.

**Задания:**
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**План проекта:**
1. Описание проекта. 
2. Подключение к базе данных.
3. Чтение таблиц из базы данных.
4. Анализ данных.
- 4.1. Количество книг, выпущенных после 1 января 2000 года.
- 4.2. Количетво обзоров и средняя оценка книги.
- 4.3. Издатели с книгами 50+ страниц.
- 4.4. Автор с самой высокой оценкой (оценок 50+).
- 4.5. Среднее количетсво обзоров от пользователей, которые поставили 50+ оценок.
5. Общий вывод.

## 2 Подключение к базе данных.

In [1]:
# импортируем библиотеки
#!/usr/bin/python3
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## 3 Чтение таблиц из базы данных.

### 3.1 Формируем sql-запрос для таблицы **`books`**, исследуем ее:

In [2]:
query = ''' SELECT *
            FROM books;
        '''
books = pd.io.sql.read_sql(query, con = engine)
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 150
#проверим наличие дубликатов:
print(f"Количество дубликатов: {books.duplicated().sum()}")
display(books.head())
display(books.info())

Количество дубликатов: 0


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

**Описание данных:**

Таблица **`books`** Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

таблица **`books`** содержит 1000 строк и 6 столбцов;

пропусков нет, дубликатов нет;

тип столбца publication_date необходимо будет преобразовать.

### 3.2 Формируем sql-запрос для таблицы **`authors`**, исследуем ее:

In [3]:
query = ''' SELECT *
            FROM authors;
        '''
authors = pd.io.sql.read_sql(query, con = engine)
#проверим наличие дубликатов:
print(f"Количество дубликатов: {authors.duplicated().sum()}")
display(authors.head())
display(authors.info())

Количество дубликатов: 0


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

**Описание данных:**

Таблица **`authors`** Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

таблица **`authors`** содержит 636 строк и 2 столбца;

пропусков нет, дубликатов нет;

тип столбцов соответствуют.

### 3.3 Формируем sql-запрос для таблицы **`publishers`**, исследуем ее:

In [4]:
query = ''' SELECT *
            FROM publishers;
        '''
publishers = pd.io.sql.read_sql(query, con = engine)
#проверим наличие дубликатов:
print(f"Количество дубликатов: {publishers.duplicated().sum()}")
display(publishers.head())
display(publishers.info())

Количество дубликатов: 0


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None

**Описание данных:**

Таблица **`publishers`** Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

таблица **`publishers`** содержит 340 строк и 2 столбца;

пропусков нет, дубликатов нет;

тип столбцов соответствуют.

### 3.4 Формируем sql-запрос для таблицы **`ratings`**, исследуем ее:

In [5]:
query = ''' SELECT *
            FROM ratings;
        '''
ratings = pd.io.sql.read_sql(query, con = engine)
#проверим наличие дубликатов:
print(f"Количество дубликатов: {ratings.duplicated().sum()}")
display(ratings.head())
display(ratings.info())

Количество дубликатов: 0


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

**Описание данных:**

Таблица **`ratings`** Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

таблица **`ratings`** содержит 6456 строк и 4 столбца;

пропусков нет, дубликатов нет;

тип столбцов соответствуют.

### 3.5 Формируем sql-запрос для таблицы **`reviews`**, исследуем ее:

In [6]:
query = ''' SELECT *
            FROM reviews;
        '''
reviews = pd.io.sql.read_sql(query, con = engine)
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 150
#проверим наличие дубликатов:
print(f"Количество дубликатов: {reviews.duplicated().sum()}")
display(reviews.head())
display(reviews.info())

Количество дубликатов: 0


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

**Описание данных:**

Таблица **`reviews`** Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

таблица **`reviews`** содержит 2793 строки и 4 столбца;

пропусков нет, дубликатов нет;

тип столбцов соответствуют.

## 4 Анализ данных.

### 4.1 Количество книг, выпущенных после 1 января 2020 года.

Работа с таблицей books.

In [7]:
query = ''' SELECT
               COUNT(title) AS cnt 
            FROM
                books
            WHERE
                publication_date :: date > '2000-01-01';
        '''

books = pd.io.sql.read_sql(query, con = engine)
books

,cnt
0,819


**819 книг** в нашей таблице с данными выпущены после 1 января 2000 года.

### 4.2 Количетво обзоров и средняя оценка книги.

Для этой задачи используем таблицу ratings и reviews.

In [8]:
query = '''
SELECT
    books.book_id,
    books.title,
    COUNT(DISTINCT review_id) as review_cnt,
    AVG(rating) as avg_rating
FROM 
    books
    LEFT JOIN reviews ON  reviews.book_id = books.book_id
    INNER JOIN ratings ON  ratings.book_id = books.book_id
GROUP BY
    books.book_id
ORDER BY
    review_cnt DESC
;
        '''
books = pd.io.sql.read_sql(query, con = engine)
books

,book_id,title,review_cnt,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Больше всего пользовательских обзоров (7) было написано для книги Twilight (Twilight #1)

### 4.3 Издатели с книгами 50+ страниц.

Для этой задачи будем использовать две таблицы: publisher и books.

In [9]:
query = '''
SELECT
    publisher,
    COUNT(book_id) as book_cnt
FROM
    publishers
LEFT JOIN books ON  books.publisher_id = publishers.publisher_id 
WHERE 
    num_pages > 50
GROUP BY
    publisher
ORDER BY
    book_cnt DESC
LIMIT 1;
'''
books = pd.io.sql.read_sql(query, con = engine)
books

,publisher,book_cnt
0,Penguin Books,42


Издательство **Penguin Books** выпустило больше всего книг с 50+ страницами.

### 4.4 Автор с самой высокой оценкой (оценок 50+).

In [10]:
query = '''
SELECT
    author,
    AVG(sub.avg_rating) as total_avg_rating
FROM
    (SELECT
        author,
        books.book_id,
        AVG(rating) as avg_rating
    FROM 
        authors
    LEFT JOIN books ON books.author_id = authors.author_id
    LEFT JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        author,
        books.book_id
    HAVING
        COUNT(rating) > 50) as sub
GROUP BY
    author
ORDER BY
    total_avg_rating DESC
LIMIT 1;
'''
books = pd.io.sql.read_sql(query, con = engine)
books

,author,total_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


**J.K. Rowling/Mary GrandPré** - автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.

### 4.5 Среднее количетсво обзоров от пользователей, которые поставили 50+ оценок.

In [11]:
query = '''
SELECT
    ROUND(AVG(sub_text_cnt.text_cnt)) AS avg_text_cnt
FROM
    (SELECT
        COUNT(text) as text_cnt
    FROM 
        reviews
    WHERE username IN
        (SELECT username
        FROM 
            ratings
        GROUP BY
            username
        HAVING
            COUNT(ratings) > 50)
    GROUP BY 
            username
    ) AS sub_text_cnt
;
'''
books = pd.io.sql.read_sql(query, con = engine)
books

,avg_text_cnt
0,24.0


Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок - **24**.

## 5 Общий вывод.

На основы исследования можно сделать следующие выводы:
1. **819 книг** выпущено после 1 января 2000 года.
2. Больше всего пользовательских обзоров (7) было написано для книги **Twilight (Twilight #1)**.
3. **Penguin Books** - издательство, которое выпустило наибольшее число книг (42 книги) толще 50 страниц.
4. **J.K. Rowling/Mary GrandPré** - автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.
5. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - **24**.

Таким образом, видим, что потребность у клиентов в продукте есть, больше всего пользователй интересует книги формата аналогичного книги **Twilight**, книги печатающиеся в издательстве
**Penguin Books**, а так же книги автора **J.K. Rowling/Mary GrandPré**. Исходя из этих даных, можем предположить что аудитория продукта молодая, как раз та аудитория котороя активно пользуется приложениями в смартфонах. 